# Preprocess

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os


In [ ]:
train_label=pd.read_csv('/content/new_train.csv')
val_label=pd.read_csv('/content/new_val.csv')

In [ ]:
labels = pd.read_csv('/content/new_labels.csv')
labels

In [ ]:
# input_size=224
path='/content/all_img/hackathon/'
images=[]
label_split=[]
for i in range(len(train_label)):
  row = train_label.iloc[i]
  img = cv2.imread(path+row['filename'])

  label = sorted(row['labels'].split())
  label_split.append(label)
  images.append(img)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
encoding = MultiLabelBinarizer()
lab = label_split + [list([str(x) for x in list(labels['id'])])]
label_onehot = encoding.fit_transform(lab)
label_onehot

In [ ]:
a=label_onehot.tolist()
del a[-1]
label_onehot = np.array(a)
label_onehot.shape

In [ ]:
# classes = list(encoding.classes_)
#classes from multilabelbinarizer
classes = ['0','1','10','100','101','102','103','104','105','106','107','108','109','11','110','111','112','113','114','115','116','117','118','119','12','120','121','122','123','124','125','126','127','128','129','13','130','131','132','133','134','135','136','137','138','139','14','140','141','142','143','144', '145','146','147', '148','149','15','150','151','152','153','154', '155','156','157','158','159','16', '160','161','162','163','164','165','166','167','168','169','17','170','171','172','173','174','175','176','177','178','179','18','180','181','182','183','184','185','186','187','188','189','19','190','191','192','193','194','195','196','197','198','199','2','20','200','201','202','203','204','205', '206','207','208','209','21','210','211','212','213', '214','215','216','217','218','219','22','220','221','222','223', '224','225','226','227','228', '229','23','230','231','232','233','234','235','236','237','238','239','24','240','241','242','243','244','245','246','247','248','249','25','250','251','252','253','254','255','256','257','258','259','26','260','261','262','263','264', '265','266','267','268','269','27','270','271','272','273','274','275','276','277','278','279','28','280','281','282','283','284','285','286','287','288','289', '29','290','291','292','293','294','295','296','297','298','299','3','30','300','301','31','32','34','35','36','37','38','39','4','40','41','42','43','44','45','46','47','48','49','5','50','51','52','53','54','55','56','57','58','59','6','60','61','62','63','64','65','66','67','68','69','7','70','71','72','73','74','75','76','77','78','79','8','80','81','82','83','84','85','86','87','88','89','9','90','91','92','93','94','95','96','97', '98','99']


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.
                            featurewise_center=True,
                            featurewise_std_normalization=True,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True)

train_generator=datagen.flow_from_dataframe(dataframe=train_label, directory=path, x_col="filename", y_col="labels", 
                                            subset="training", batch_size=256, seed=42,classes=classes, shuffle=True, 
                                            class_mode="categorical", target_size=(224,224))

valid_generator=datagen.flow_from_dataframe( dataframe=val_label, directory=path, x_col="filename", y_col="labels", 
                                            subset="training", batch_size=256, seed=42,classes=classes, shuffle=True, 
                                            class_mode="categorical", target_size=(224,224))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=df_test, directory="./hackathon/", x_col="filename", 
                                                y_col=None, batch_size=256, seed=42, 
                                                shuffle=False, class_mode=None, target_size=(224,224))

# Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
IMG_SIZE=224
out_dim = 302

pretrain = tf.keras.applications.EfficientNetB0(include_top=False,weights="imagenet")

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = pretrain(inputs)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(out_dim, activation='sigmoid')(x)

model = tf.keras.Model(inputs, x)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=["accuracy"])
model.summary()

In [ ]:
filepath='/content/my_best_model.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_loss')
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1,min_delta=0.001)

model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate=1e-2), loss = 'binary_crossentropy',metrics = ['acc'])
batch_size = 64
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,callbacks=[callbacks,checkpoint],
                    epochs=30)

# Predict

In [ ]:
##Predict and submission on kaggle ##
proba = model.predict(test_generator)
submission=[]
for i,p in enumerate(proba):
  index_p=np.where(proba[i] >0.3)
  class_pred=[]
  for j in index_p[0]:
    k = int(classes[int(j)])
    # print(j, labels['label'].iloc[j])
    class_pred.append(k)
    submission.append(sorted(class_pred))